In [1]:
import numpy as np
import pandas as pd
import math
import random

Leer XP_Table

In [2]:
## Solo para Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_data = '/content/drive/MyDrive/Python-Proyectos/FFI/Data/'

In [4]:
df_XP_Table = pd.read_csv(path_data+'XP_Solo_Table.csv',sep=',')

In [5]:
df_XP_Table.tail()

,Nv,XP
94,95,923638
95,96,938137
96,97,952637
97,98,967137
98,99,981637


In [6]:
df_weapons = pd.read_excel(path_data+'FFI_BD_Weapons.xlsx', index_col = 'New Name')

In [7]:
df_weapons.head(4)

,Old Name,Price,Atk,Acc,Crit,Equipped by,Buy,Find,Win,Special,Type,Strong vs,Element,Cast,Inflicts,Stats,Special Effect Before Attack,Special Effect After Attack
New Name,,,,,,,,,,,,,,,,,,
Staff,Wooden staff,5,6,0,3,"Fi,Kn,Ni,RM,RW,BB,Ma,WM,WW,BM,BW",Cornelia,several,-,False,staff,-,-,-,-,-,-,-
Power staff,Power staff,24690,12,0,28,"Fi,Kn,Ni,RM,RW,BB,Ma,WM,WW,BM,BW",-,Western Keep,-,False,staff,-,-,-,-,-,-,-
Rapier,Rapier,10,9,0.05,4,"Fi,Kn,Th,Ni,RM,RW",Cornelia,-,-,False,sword,-,-,-,-,-,-,-
Scimitar,Scimtar,200,10,0.1,8,"Fi,Kn,Th,Ni,RM,RW",Pravoka,-,-,False,sword,-,-,-,-,-,-,-


In [8]:
data_ini_stat = {'HP':[35, 30, 33, 25, 33, 30],
        'FRZ':[10, 5, 5, 3, 12, 5],
        'AGI':[8, 15, 5, 5, 5, 10],
        'INT':[1, 1, 15, 20, 1,10],
        'RES':[15, 5, 8, 2, 10, 5],
        'SUE':[8, 15, 5, 10, 5, 5],
        'P_HP':[0.5, 0.35, 0.3, 0.25, 0.4, 0.25],
        'P_F':[0.75, 0.625, 0.375, 0.32, 0.5, 0.5],
        'P_A':[0.625, 0.625, 0.375, 0.32, 0.5, 0.375],
        'P_I':[0.32, 0.375, 0.625, 0.75, 0.5, 0.5],
        'P_R':[0.5, 0.375, 0.375, 0.32, 0.75, 0.5],
        'P_S':[0.5, 0.75, 0.375, 0.32, 0.625, 0.5],
        'Ini_Pun':[7, 10, 4, 6, 5, 9],
        'Mul_Lvl_Pun':[3, 5, 1, 2, 3, 3],
        'Ini_Eva':[45, 43, 48, 48, 45, 43]
        }
df_ini_stat = pd.DataFrame(data_ini_stat, index=['Fi/Kn','Th/Ni','Wm/Wz','Bm/Bz','Bb/Ma','Rm/Rz'])
df_ini_stat

,HP,FRZ,AGI,INT,RES,SUE,P_HP,P_F,P_A,P_I,P_R,P_S,Ini_Pun,Mul_Lvl_Pun,Ini_Eva
Fi/Kn,35,10,8,1,15,8,0.50,0.750,0.625,0.320,0.500,0.500,7,3,45
Th/Ni,30,5,15,1,5,15,0.35,0.625,0.625,0.375,0.375,0.750,10,5,43
Wm/Wz,33,5,5,15,8,5,0.30,0.375,0.375,0.625,0.375,0.375,4,1,48
Bm/Bz,25,3,5,20,2,10,0.25,0.320,0.320,0.750,0.320,0.320,6,2,48
Bb/Ma,33,12,5,1,10,5,0.40,0.500,0.500,0.500,0.750,0.625,5,3,45
Rm/Rz,30,5,10,10,5,5,0.25,0.500,0.375,0.500,0.500,0.500,9,3,43


Fi/Kn - FIGHTER / KNIGHT classes\
Th/Ni - THIEF / NINJA classes\
Wm/Wz - WHITE MAGE / WHITE WIZARD classes\
Bm/Bz - BLACK MAGE / BLACK WIZARD classes\
Bb/Ma - BLACK BELT / MASTER classes\
Rm/Rz - RED MAGE / RED WIZARD classes

In [9]:
def exactDualType(tipo):
    if tipo == 'Gue' or tipo == 'War' or tipo == '' or tipo == 'Fi' or tipo == 'Kn':
        return 'Fi/Kn'
    elif tipo == 'Lad' or tipo == 'Thi' or tipo == 'Th' or tipo == 'Ni':
        return 'Th/Ni'
    elif tipo == 'MB' or tipo == 'WM' or tipo == 'WW':
        return 'Wm/Wz'
    elif tipo == 'MN' or tipo == 'BM' or tipo == 'BW':
        return 'Bm/Bz'
    elif tipo == 'Mon' or tipo == 'BB' or tipo == 'Ma':
        return 'Bb/Ma'
    elif tipo == 'MR' or tipo == 'RM' or tipo == 'RW':
        return 'Rm/Rz'

def exactType(tipo):
    if tipo == 'Gue' or tipo == '':
        return 'Fi'
    elif tipo == 'War':
        return 'Kn'
    elif tipo == 'Lad' or tipo == 'Thi':
        return 'Th'
    elif tipo == 'Nin':#Ninja
        return 'Ni'
    elif tipo == 'MB' or tipo == 'WM':#Mago Blanco, White Mage
        return 'WM'
    elif tipo == 'WW':#White Wizard
        return 'WW'
    elif tipo == 'MN' or tipo == 'BM':#Mago Negro, Black Mage
        return 'BM'
    elif tipo == 'BW':#Black Wizard
        return 'BW'
    elif tipo == 'Mon' or tipo == 'BB':#Monje, Black Belt 
        return 'BB'
    elif tipo == 'Ma':#Master
        return 'Ma'
    elif tipo == 'MR' or tipo == 'RM':#Mago Rojo, Red Mage
        return 'RM'
    elif tipo == '':#Red Wizard
        return 'RW'

In [75]:
# Funciones Globales

def xp_Lv(Nxt_Lv, max_Lv):
    if Nxt_Lv > max_Lv:
        return math.inf
    else:
        return df_XP_Table.iloc[Nxt_Lv-1,1]

def elem_in_array(arr):
    return arr.split(',')

def arma_inicial(tipo):
    if tipo == 'Fi' or tipo == 'Th' or tipo == 'BM' or tipo == 'RM':
        return Arma('Knife')
    elif tipo == 'WM' or tipo == 'BB':
        return Arma('Staff')

def str_to_dict(s):
    if s == '-':
        return s
    else:
        s = s.replace("{", "")
        s = s.replace("}", "")
        arr = s.split(", ")
        miDic={}
        for elem in arr:
            val = elem.split(":")
            miDic[val[0]] = int(val[1])
            #print(type(miDic[val[0]]))
        return miDic

def getAllStats(dic):#dic es dict, excepto si es '-'
    aux = {'FRZ':0, 'AGI':0, 'INT':0, 'RES':0, 'SUE':0, 'Eva':0}
    if dic == '-':
        return aux
    else:
        #print('getAllStats: dic:', dic)
        #print('tipo dic[FRZ]: ', type(dic['FRZ']))
        if 'FRZ' in dic:
            aux['FRZ'] = dic['FRZ']
        if 'AGI' in dic:
            aux['AGI'] = dic['AGI']
        if 'INT' in dic:
            aux['INT'] = dic['INT']
        if 'RES' in dic:
            aux['RES'] = dic['RES']
        if 'SUE' in dic:
            aux['SUE'] = dic['SUE']
        if 'Eva' in dic:
            aux['Eva'] = dic['Eva']
        #print('getAllStats: aux:', aux)
        return aux

In [77]:
class Arma:
    def __init__(self, name):
        self.nombre = name
        if name != 'Hands':
            self.price = df_weapons['Price'][self.nombre]
            self.value = 0
            if isinstance(self.price, int):
                self.value = self.price//2
            self.atk = df_weapons['Atk'][self.nombre]
            self.acc = df_weapons['Acc'][self.nombre]
            self.crit = df_weapons['Crit'][self.nombre]
            self.equip_by = elem_in_array(df_weapons['Equipped by'][self.nombre])
            self.buy = df_weapons['Buy'][self.nombre]
            self.find = df_weapons['Find'][self.nombre]
            self.win = df_weapons['Win'][self.nombre]
            self.special = df_weapons['Special'][self.nombre]
            self.tipo = df_weapons['Type'][self.nombre]
            self.str_vs = df_weapons['Strong vs'][self.nombre]
            self.elemento = df_weapons['Element'][self.nombre]
            self.cast = df_weapons['Cast'][self.nombre]
            self.inflige = df_weapons['Inflicts'][self.nombre]
            self.stats = str_to_dict(df_weapons['Stats'][self.nombre])
            self.allStats = getAllStats(self.stats)
            self.seba = df_weapons['Special Effect Before Attack'][self.nombre]
            self.seaa = df_weapons['Special Effect After Attack'][self.nombre]
        else:
            self.price,self.value,self.atk,self.acc,self.crit,self.equip_by = 0,0,0,0,0,'All clases'
            self.buy,self.find,self.win,self.special,self.tipo,self.str_vs = '-','-','-',False,'-','-'
            self.elemento,self.cast,self.inflige,self.stats,self.seba,self.seaa = '-','-','-','-','-','-'
    def mostrar_Info(self):
        print("Precio:", self.price, '\t', type(self.price))
        print("Atk:", self.atk, '\t', type(self.atk))
        print("Acc:", self.acc, '\t', type(self.acc))
        print("Crit:", self.crit, '\t', type(self.crit))
        print("Equipped by:", self.equip_by, '\t', type(self.equip_by))
        print("Buy:", self.buy, '\t', type(self.buy))
        print("Find:", self.find, '\t', type(self.find))
        print("Win:", self.win, '\t', type(self.win))
        print("Special:", self.special, '\t', type(self.special))
        print("Type:", self.tipo, '\t', type(self.tipo))
        print("Strong vs:", self.str_vs, '\t', type(self.str_vs))
        print("Element:", self.elemento, '\t', type(self.elemento))
        print("Cast:", self.cast, '\t', type(self.cast))
        print("Inflicts:", self.inflige, '\t', type(self.inflige))
        print("Stats:", self.stats, '\t', type(self.stats))
        print("Special Effect Before Attack:", self.seba, type(self.seba))
        print("Special Effect After Attack:", self.seaa, type(self.seaa))

In [89]:
class Personaje:
    def __init__(self, tipo = ''):
        self.dualtype = exactDualType(tipo)
        self.tipo = exactType(tipo)
        self.Lv = 1
        self.XP = 0
        self.alive = True
        self.MAX_LV = df_XP_Table.shape[0] #99
        self.XP_limit0 = xp_Lv(self.Lv, self.MAX_LV)
        self.XP_limit1 = xp_Lv(self.Lv+1, self.MAX_LV)

        self.MAX_HP_base = df_ini_stat['HP'][self.dualtype]
        self.Por_aum_HP = 0.0
        self.MAX_HP = math.trunc(self.MAX_HP_base * (1 + self.Por_aum_HP))
        self.HP = self.MAX_HP_base

        self.FRZ_base = df_ini_stat['FRZ'][self.dualtype]
        self.AGI_base = df_ini_stat['AGI'][self.dualtype]
        self.INT_base = df_ini_stat['INT'][self.dualtype]
        self.RES_base = df_ini_stat['RES'][self.dualtype]
        self.SUE_base = df_ini_stat['SUE'][self.dualtype]
        self.FRZ_total = self.FRZ_base
        self.AGI_total = self.AGI_base
        self.INT_total = self.INT_base
        self.RES_total = self.RES_base
        self.SUE_total = self.SUE_base

        self.Atq_base = self.FRZ_total//2
        self.Pun_base = df_ini_stat['Ini_Pun'][self.dualtype] + self.AGI_total + (self.Lv * df_ini_stat['Mul_Lvl_Pun'][self.dualtype])
        self.Def_base = 0
        if self.dualtype == 'Bb/Ma':
            self.Def_base = (self.RES_total // 4)+3 + (((self.Lv-1) // 10) * 3)
        #self.Def_base = self.dualtype == 'Bb/Ma'? (self.RES_total // 4)+3 + (((self.Lv-1) // 10) * 3) : 0
        self.Eva_base = df_ini_stat['Ini_Eva'][self.dualtype] + 2*self.AGI_total
        self.Atq_total = self.Atq_base
        self.Pun_total = self.Pun_base
        self.Def_total = self.Def_base
        self.Eva_total = self.Eva_base

        self.weapon = arma_inicial(self.tipo)

        self.P_HP = df_ini_stat['P_HP'][self.dualtype]
        self.P_F = df_ini_stat['P_F'][self.dualtype]
        self.P_A = df_ini_stat['P_A'][self.dualtype]
        self.P_I = df_ini_stat['P_I'][self.dualtype]
        self.P_R = df_ini_stat['P_R'][self.dualtype]
        self.P_S = df_ini_stat['P_S'][self.dualtype]
    def mostrar(self, show_title = True):
        if show_title:
            print('Nv.\t' + 'HP\t' + 'FRZ\t' + 'AGI\t' + 'INT\t' + 'RES\t' + 'SUE')
        print(str(self.Lv) + '\t' + str(self.MAX_HP_base) + '\t' + str(self.FRZ_base) + '\t' + str(self.AGI_base) + '\t' + str(self.INT_base) + '\t' + str(self.RES_base) + '\t' + str(self.SUE_base))
    def mostrar_Total(self, show_title = True):
        if show_title:
            print('Nv.\t' + 'HP\t' + 'FRZ\t' + 'AGI\t' + 'INT\t' + 'RES\t' + 'SUE')
        print(str(self.Lv) + '\t' + str(self.MAX_HP) + '\t' + str(self.FRZ_total) + '\t' + str(self.AGI_total) + '\t' + str(self.INT_total) + '\t' + str(self.RES_total) + '\t' + str(self.SUE_total))
    def actualizar_el_resto(self):
        self.Lv += 1
        self.XP_limit0 = xp_Lv(self.Lv, self.MAX_LV)
        self.XP_limit1 = xp_Lv(self.Lv+1, self.MAX_LV)
    def prob_aum(self, prob):
        if prob > random.random():
            return 1
        else:
            return 0
    def actualizar_stats(self, show_title):
        self.MAX_HP_base = self.MAX_HP_base + math.trunc(self.RES/4) + 1 + self.prob_aum(self.P_HP)*random.randint(20, 25)
        self.FRZ_base = self.FRZ_base + self.prob_aum(self.P_F)
        self.AGI_base = self.AGI_base + self.prob_aum(self.P_A)
        self.INT_base = self.INT_base + self.prob_aum(self.P_I)
        self.RES_base = self.RES_base + self.prob_aum(self.P_R)
        self.SUE_base = self.SUE_base + self.prob_aum(self.P_S)
        #self.mostrar(show_title)  #
    def Lv1UP(self, show_title = False):
        self.actualizar_el_resto()
        self.actualizar_stats(show_title)
    def actualizar_Lv(self, show_title):
        while 1:
            if (self.XP >= self.XP_limit0 and self.XP < self.XP_limit1):
                break
            self.Lv1UP(show_title)
    def subirXP(self, cant, show_title):
        self.XP += cant
        if (self.XP < self.XP_limit0 or self.XP >= self.XP_limit1):
            self.actualizar_Lv(show_title)    #OK, sube 1 o más Lv
    def mostrar2(self):
        print('Nv.\t' + 'XP\t' + 'XP_limit0' + 'XP_limit1')
        print(str(self.Lv) + '\t' + str(self.XP) + '\t' + str(self.XP_limit0) + '\t' + str(self.XP_limit1))
    def valid_dep(self):
        atq_wea_equip = df_weapons['Atk'][self.weapon.nombre] #ataque del arma equipada en BD
        acc_wea_equip = df_weapons['Acc'][self.weapon.nombre] #accuracy del arma equipada en BD
        if isinstance(atq_wea_equip, str):#si atk es str
            if self.weapon.atk == 'HP/10':
                self.weapon.atk = self.MAX_HP // 10
        if isinstance(acc_wea_equip, str):#si acc es str
            if self.weapon.acc == 'HP/10':
                self.weapon.acc = self.MAX_HP // 10
    def actualizar_weapon_stats(self):#add weapon, desde Hands
        if self.weapon.nombre == 'Hands':
            self.actualizar_weapon_remove()
            return
        wStats = getAllStats(self.weapon.stats)
        #print('wStats:', wStats)
        self.valid_dep() # de armas a personaje (Ultima Weapon)
        self.FRZ_total = self.FRZ_base + wStats['FRZ']
        self.AGI_total = self.AGI_base + wStats['AGI']
        self.INT_total = self.INT_base + wStats['INT']
        self.RES_total = self.RES_base + wStats['RES']
        self.SUE_total = self.SUE_base + wStats['SUE']
        self.Eva_total = self.Eva_base + wStats['Eva']
        self.Atq_total = self.Atq_base + self.weapon.atk
        self.Pun_total = math.trunc(self.Pun_base + 100*self.weapon.acc)
    def actualizar_weapon_remove(self):#remove weapon
        self.FRZ_total = self.FRZ_base
        self.AGI_total = self.AGI_base
        self.INT_total = self.INT_base
        self.RES_total = self.RES_base
        self.SUE_total = self.SUE_base
        self.Atq_total = self.Atq_base
        self.Pun_total = self.Pun_base
        self.Def_total = self.Def_base
        self.Eva_total = self.Eva_base
    def showAllStats(self):
        tipo = self.tipo
        arma = self.weapon
        nom_arma = arma.nombre
        print('\t'+tipo+' - Hands'+'\t'+nom_arma+'\t'+tipo+' - '+nom_arma)
        print('HP\t'+str(self.MAX_HP_base)+'\t\t\t\t'+str(self.MAX_HP))
        print('FRZ\t'+str(self.FRZ_base)+'\t\t'+str(arma.allStats['FRZ'])+'\t\t'+str(self.FRZ_total))
        print('AGI\t'+str(self.AGI_base)+'\t\t'+str(arma.allStats['AGI'])+'\t\t'+str(self.AGI_total))
        print('INT\t'+str(self.INT_base)+'\t\t'+str(arma.allStats['INT'])+'\t\t'+str(self.INT_total))
        print('RES\t'+str(self.RES_base)+'\t\t'+str(arma.allStats['RES'])+'\t\t'+str(self.RES_total))
        print('SUE\t'+str(self.SUE_base)+'\t\t'+str(arma.allStats['SUE'])+'\t\t'+str(self.SUE_total))
        print('Atq\t'+str(self.Atq_base)+'\t\t'+str(arma.atk)+'\t\t'+str(self.Atq_total))
        print('Pun\t'+str(self.Pun_base)+'\t\t'+str(arma.acc)+'\t\t'+str(self.Pun_total))
        print('Def\t'+str(self.Def_base)+'\t\t\t\t'+str(self.Def_total))
        print('Eva\t'+str(self.Eva_base)+'\t\t'+str(arma.allStats['Eva'])+'\t\t'+str(self.Eva_total))

In [90]:
class ArrPersonaje:
    def __init__(self):
        self.n = 0
        self.arr = []
    def addPer(self, Per):
        self.n += 1
        self.arr.append(Per)
    def addArrPer(self, ArrPer):
        self.n = len(ArrPer)
        for per in ArrPer:
            self.arr.append(per)
    def mostrarTodo(self):
        for per in self.arr:
            per.mostrar()
    def mostrarTodo2(self):
        for per in self.arr:
            per.mostrar2()

    def arrAlive(self):
        aux = []
        for per in self.arr:
            if per.alive:
                aux.append(per)
        return aux
    def subirXP_gru(self, cant):
        auxArrAlive = self.arrAlive()
        cant_XP_One = math.trunc(cant/len(auxArrAlive))
        for per in auxArrAlive:
            per.subirXP(cant_XP_One, False)

In [50]:
# Creación del grupo
Escanor = Personaje('Gue')
Arthas = Personaje('Lad')
Merlin = Personaje('WM')
Lich = Personaje('BM')
Jackie = Personaje('BB')
Gill = Personaje('RM')
party = [Escanor,Arthas,Merlin,Lich,Jackie,Gill]
Party = ArrPersonaje()
Party.addArrPer(party)

In [ ]:
]Party.mostrarTodo2()

In [ ]:
for per in party:
    per.mostrar()

Nv.	HP	FRZ	AGI	INT	RES	SUE
250	8030	195	159	86	145	135
Nv.	HP	FRZ	AGI	INT	RES	SUE
250	4897	167	163	92	90	209
Nv.	HP	FRZ	AGI	INT	RES	SUE
250	4944	88	99	168	88	93
Nv.	HP	FRZ	AGI	INT	RES	SUE
250	4341	70	84	220	80	99


In [ ]:
#elevarlos al nivel 250 para ver como serían sus stats
for i in range(249):
    for per in party:
        per.Lv1UP()

In [ ]:
Party.arr[0].alive = False

In [ ]:
x = 20
y = '?'

print(isinstance(x, int))

True


In [ ]:
print(type(df_weapons['Price']['War Hammer']))

<class 'int'>


In [ ]:
df_weapons['Price']#['War Hammer']

New Name
Wooden staff             5
Power staff          24690
Rapier                  10
Scimitar               200
Battle axe             550
                     ...  
Assassin dagger          ?
War Hammer           38000
Lust Dagger              ?
Barbarian's Sword        ?
Golden Staff             ?
Name: Price, Length: 67, dtype: object

In [ ]:
df_weapons['Price']['War Hammer']

38000

In [ ]:
df_ini_stat['P_HP']#['Fi/Kn']

Fi/Kn    0.50
Th/Ni    0.35
Wm/Wz    0.30
Bm/Bz    0.25
Bb/Ma    0.40
Rm/Rz    0.25
Name: P_HP, dtype: float64

In [ ]:
df_weapons.loc['War Hammer']['Price']

38000

In [ ]:
df_weapons['Price']['War Hammer']

38000

In [17]:
#arma_1 = Arma('Rapier')
#arma_1 = Arma('Ultima weapon')
arma_1 = Arma('Ragnarok')
#arma_1 = Arma('Hands')

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [18]:
arma_1.mostrar_Info()

Precio: ? 	 <class 'str'>
Atk: 55 	 <class 'int'>
Acc: 0.4 	 <class 'float'>
Crit: 39 	 <class 'int'>
Equipped by: ['Kn'] 	 <class 'list'>
Buy: - 	 <class 'str'>
Find: - 	 <class 'str'>
Win: Shinryu 	 <class 'str'>
Special: True 	 <class 'numpy.bool_'>
Type: sword 	 <class 'str'>
Strong vs: - 	 <class 'str'>
Element: - 	 <class 'str'>
Cast: Flare 	 <class 'str'>
Inflicts: - 	 <class 'str'>
Stats: {'FRZ': 7, 'RES': 7, 'AGI': 3, 'Eva': 30, 'INT': 7} 	 <class 'dict'>
Special Effect Before Attack: - <class 'str'>
Special Effect After Attack: - <class 'str'>


In [ ]:
Escanor.tipo

'Fi'

In [ ]:
if Escanor.tipo in arma_1.equip_by:
    print('Si')
else:
    print('No')

Si


In [ ]:
type(arma_1.find)

str

In [51]:
Escanor.tipo = 'Kn'
Escanor.weapon = Arma('Ragnarok')
Escanor.actualizar_weapon_stats()
print(Escanor.weapon)

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
getAllStats: dic: {'FRZ': 7, 'RES': 7, 'AGI': 3, 'Eva': 30, 'INT': 7}
tipo dic[FRZ]:  <class 'int'>
getAllStats: aux: {'FRZ': 7, 'AGI': 3, 'INT': 7, 'RES': 7, 'SUE': 0, 'Eva': 30}
getAllStats: dic: {'FRZ': 7, 'RES': 7, 'AGI': 3, 'Eva': 30, 'INT': 7}
tipo dic[FRZ]:  <class 'int'>
getAllStats: aux: {'FRZ': 7, 'AGI': 3, 'INT': 7, 'RES': 7, 'SUE': 0, 'Eva': 30}
wStats: {'FRZ': 7, 'AGI': 3, 'INT': 7, 'RES': 7, 'SUE': 0, 'Eva': 30}


In [20]:
Escanor.mostrar()
Escanor.mostrar_Total()

Nv.	HP	FRZ	AGI	INT	RES	SUE
1	35	10	8	1	15	8
Nv.	HP	FRZ	AGI	INT	RES	SUE
1	35.0	10	8	1	15	8


In [ ]:
#UA = {'FRZ': 0, 'AGI': 0, 'INT': 0, 'RES': 0, 'SUE': 0, 'Eva': 0}
UA = "{FRZ: 0, AGI: 0, INT: 0, RES: 0, SUE: 0, Eva: 0}"
#UA = "{'FRZ': 0, 'AGI': 0, 'INT': 0, 'RES': 0, 'SUE': 0, 'Eva': 0}"
#print(UA['FRZ'], type(UA['FRZ']))
print(type(UA))

<class 'str'>


In [ ]:
asd = UA.replace("{", "")
asd = asd.replace("}", "")
print(asd)
arr = asd.split(", ")
print(arr)
miDic={}
for elem in arr:
    val = elem.split(":")
    #print('val[0]:', val[0], 'val[1]:', val[1])
    miDic[val[0]] = int(val[1])
    print(type(miDic[val[0]]))

FRZ: 0, AGI: 0, INT: 0, RES: 0, SUE: 0, Eva: 0
['FRZ: 0', 'AGI: 0', 'INT: 0', 'RES: 0', 'SUE: 0', 'Eva: 0']
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [ ]:
sq = "7"
iq = int(sq)
print('iq: ', iq, type(iq))

iq:  7 <class 'int'>


In [ ]:
df_weapons['Atk']['Ultima weapon']

'HP/10'

In [ ]:
df_weapons['Atk'][arma_1.nombre]

'HP/10'

In [ ]:
arma_1.atk

'HP/10'

In [ ]:
print(Escanor.Atq_base, Escanor.Atq_total)
print(Escanor.Pun_base, Escanor.Pun_total)

5.0 8.0
18 21.0


In [60]:
Escanor.showAllStats()

	Kn - Hands	Ragnarok	Kn - Ragnarok
HP	35		35.0
FRZ	10		7		17
AGI	8		3		11
INT	1		7		8
RES	15		7		22
SUE	8		0		8
Eva	61		30		91


In [34]:
UA = {'FRZ': 0, 'AGI': 0, 'INT': 0, 'RES': 0, 'SUE': 0, 'Eva': 0}
print(UA['FRZ'])

0


In [38]:
Escanor.weapon.stats

{'AGI': 3, 'Eva': 30, 'FRZ': 7, 'INT': 7, 'RES': 7}

In [91]:
# Creación del grupo
Escanor = Personaje('Gue')
Arthas = Personaje('Lad')
Merlin = Personaje('WM')
Lich = Personaje('BM')
Jackie = Personaje('BB')
Gill = Personaje('RM')
party = [Escanor,Arthas,Merlin,Lich,Jackie,Gill]
Party = ArrPersonaje()
Party.addArrPer(party)
# Cambiar de clase a Escanor, darle arma
Escanor.tipo = 'Kn'
Escanor.weapon = Arma('Ragnarok')
Escanor.actualizar_weapon_stats()
#print(Escanor.weapon)
Escanor.showAllStats()

	Kn - Hands	Ragnarok	Kn - Ragnarok
HP	35				35
FRZ	10		7		17
AGI	8		3		11
INT	1		7		8
RES	15		7		22
SUE	8		0		8
Atq	5		55		60
Pun	18		0.4		58
Def	0				0
Eva	61		30		91
